In [1]:
import pandas as pd
import re

In [2]:
houseData = pd.read_csv('housedata2022.csv')

In [3]:
housesWithMissingSize = houseData[houseData['Size'] == "?"]
housesWithMissingSize
sameNameAsHousesWithMissingSize = houseData[houseData['name'].isin(housesWithMissingSize['name'])]

for house in housesWithMissingSize.iterrows():
    houseName = house[1]['name']
    houseSize = sameNameAsHousesWithMissingSize[sameNameAsHousesWithMissingSize['name'] == houseName]['Size']
    houseData.loc[house[0], 'Size'] = houseSize.values[0]

In [4]:
housesWithMissingRoom = houseData[houseData['Room'] == "?"]
sameNameAsHousesWithMissingRoom = houseData[houseData['name'].isin(housesWithMissingRoom['name'])]
for house in housesWithMissingSize.iterrows():
    houseName = house[1]['name']
    houseRoom = sameNameAsHousesWithMissingRoom[sameNameAsHousesWithMissingRoom['name'] == houseName]['Room']
    houseData.loc[house[0], 'Room'] = houseSize.values[0]
housesWithMissingRoom

,name,Size,Room,Fee,Endprice,KvMPrice,Saledate,City,District,TypeOfProperty
6,Långängsvägen 21X,72,?,NaN,1185000,0,2 sep. 2024,Västerås kommun,Skälby,Övrigt
80,Solvägen 7,5 ru,?,Slutpris 3 200 000,3200000,0,26 jun. 2024,Hallstahammars kommun,Kolbäck,Villa
92,Usterstavägen 64,121+42,?,Slutpris 1 920 000,1920000,0,24 maj. 2024,Hallstahammars kommun,Näs,Villa
221,Rådjursvägen 19,174+20,?,Slutpris 2 750 000,2750000,0,21 aug. 2024,Enköpings kommun,Enköping,Villa
244,Hälke 1:34,1 929,?,NaN,320000,0,4 aug. 2024,Enköpings kommun,Örsundsbro,Tomt
247,Kesuddsvägen 7,1 261,?,NaN,2475000,0,29 jul. 2024,Enköpings kommun,Bredsand,Tomt
248,Eknäs,2 505,?,NaN,660000,0,29 jul. 2024,Enköpings kommun,Vallby,Tomt
278,Holma 223,1 357,?,NaN,335000,0,7 maj. 2024,Kungsörs kommun,Holma,Fritidshus


In [5]:
removeindexSize = houseData[houseData['Size'] == "?"].index
removeindexRoom = houseData[houseData['Room'] == "?"].index
removeindex = removeindexSize.union(removeindexRoom)
houseData.drop(removeindex, inplace=True)
print(f"removed {len(removeindex)} rows with missing or room")

removed 8 rows with missing or room


In [6]:
def removePlusAndHalf(row):
    if '+' in row['Size']:
        row['Size'] = row['Size'].split('+')[0]
    if '.' in row['Size']:
        row['Size'] = row['Size'].split('.')[0]
    if '.' in row['Room']:
        print(row['Room'])
        row['Room'] = row['Room'].split('.')[0]
        print(f"after: {row['Room']}")
    return row

houseData = houseData.apply(removePlusAndHalf, axis=1)
houseData['Room'] = houseData['Room'].astype(int)
houseData['Size'] = houseData['Size'].astype(int)

    

1.5
after: 1
2.5
after: 2
3.5
after: 3
1.5
after: 1


In [7]:
roomsWhereSquareMetersIsSmaller = houseData[houseData['Size'] < houseData['Room']]
for house in roomsWhereSquareMetersIsSmaller.iterrows():
    index = house[0]
    size = houseData.loc[index, 'Room']
    houseData.loc[index, 'Room'] = house[1]['Size']
    houseData.loc[index, 'Size']  = size
roomsWhereSquareMetersIsSmaller = houseData[houseData['Size'] < houseData['Room']]
roomsWhereSquareMetersIsSmaller

,name,Size,Room,Fee,Endprice,KvMPrice,Saledate,City,District,TypeOfProperty


In [8]:
houseData = houseData.fillna(0)
houseData = houseData.replace('?', 0)
    

In [9]:
def calculateKvmPrice(row):
    return row['Endprice'] / row['Size']

for index, row in houseData.iterrows():
    houseData.loc[index, 'KvMPrice'] = calculateKvmPrice(row)

houseData

/tmp/ipykernel_139218/917039083.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '44495.4128440367' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  houseData.loc[index, 'KvMPrice'] = calculateKvmPrice(row)


,name,Size,Room,Fee,Endprice,KvMPrice,Saledate,City,District,TypeOfProperty
0,Stadsträdgårdsgatan 12,109,5,5 373,4850000,44495.412844,4 sep. 2024,Västerås kommun,Stadsträdgården / Framnäs,Lägenhet
1,Knutsgatan 10A,103,4,6 160,3150000,30582.524272,3 sep. 2024,Västerås kommun,Centralt,Lägenhet
2,Stentorpsgatan 22B,26,1,2 048,390000,15000.000000,3 sep. 2024,Västerås kommun,Malmaberg,Lägenhet
3,Askholmsvägen 20A,133,5,0,9800000,73684.210526,3 sep. 2024,Västerås kommun,Tidö-Lindö,Villa
4,Iduns väg 5,153,5,0,3350000,21895.424837,3 sep. 2024,Västerås kommun,Irsta,Villa
...,...,...,...,...,...,...,...,...,...,...
295,Mästaregatan 18A,56,2,3 012,795000,14196.428571,3 mar. 2024,Kungsörs kommun,Kungsör,Lägenhet
296,Sågvägen 39,85,3,0,1400000,16470.588235,29 feb. 2024,Kungsörs kommun,Kungsör,Villa
297,Vallgatan 3,71,2,0,1200000,16901.408451,27 feb. 2024,Kungsörs kommun,Kungsör,Villa
298,Birkagatan 9B,80,4,0,1525000,19062.500000,27 feb. 2024,Kungsörs kommun,Kungsör,Villa


In [10]:
def convertDistrictToLower(row):
    row['District'] = row['District'].lower().capitalize()
    return row

def removeDouble(row):
    row['District'] = row['District'].split(',')[0]
    return row

def checkIfPostNumber(row):
    pattern = r"\d{5}"
    if re.search(pattern, row['District']):
        print(row['District'])

def replaceTypes(row):
    row['TypeOfProperty'] = row['TypeOfProperty'].lower()

    if "rätt" in row['TypeOfProperty']:
        row['TypeOfProperty'] = 'Bostadsrättslägenhet'
    elif any(substring in row['TypeOfProperty'] for substring in ['par', 'rad', 'kedjehus']):
        row['TypeOfProperty'] = 'Radhus'
    elif "illa" in row['TypeOfProperty']:
        row['TypeOfProperty'] = 'Villa'
    elif "ård" in row['TypeOfProperty']:
        row['TypeOfProperty'] = 'Gård'
    elif "hus" in row['TypeOfProperty']:
        row['TypeOfProperty'] = 'Fritidshus'
    else:
        row['TypeOfProperty'] = 'Other'
        
    return row

houseData = houseData.apply(convertDistrictToLower, axis=1)
houseData = houseData.apply(removeDouble, axis=1)
houseData.apply(checkIfPostNumber, axis=1)
houseData = houseData.apply(replaceTypes, axis=1)

houseData.to_csv('cleanedHouseData2022.csv', index=False)
